In [ ]:
#Required libraries:
# !pip install pyserial
# !pip install pandas
# !pip install pygame
# !pip install face_recognition
# !pip install opencv-python
# !pip install SimpleCV
# !pip install matplotlib

In [1]:
# import pygame
# import pygame.camera
import os
import face_recognition
import serial
import ftplib
import time
import cv2
from matplotlib import pyplot as plt
# from SimpleCV import Camera
import time

In [6]:
#References: 
# - https://github.com/ageitgey/face_recognition/blob/master/examples/facerec_from_webcam_faster.py
# - https://pypi.org/project/face-recognition/
# - https://github.com/nicknochnack/OpenCV-Basics/blob/main/4.%20Accessing%20Your%20Webcam.ipynb

def start(comport):
    ser = serial.Serial(comport, 9600)
    receivestring = ser.readline() # ignore the first read in
    while True:
        receivestring = ser.readline().decode("utf-8").strip() # read one line and
        time.sleep(1)  
        break
    return receivestring

def uniquify(path):
    filename, extension = os.path.splitext(path)
    counter = 1
    while os.path.exists(path):
        path = filename + " (" + str(counter) + ")" + extension
        counter += 1
    return path

def captureimage():
    ramp_frames = 30 
    cap = cv2.VideoCapture(0)
    for i in range(ramp_frames):
         temp = cap.read()
    camera_capture,img = cap.read()
    cv2.imwrite("newphoto.jpg",img)
    cap.release()
        
def uploadFTP(filenameImage,ftp):    
    with open(filenameImage, "rb") as file:
        ftp.storbinary(f"STOR {filenameImage}", file)

def openDoor(outcomport):
    ser = serial.Serial(outcomport, 9600, timeout = 1, write_timeout= 1)
    ser.write(b'1');
        
def facialRecognitionModel(filenameImage,ftp,outcomport):
    dir_path = list(ftp.mlsd())

    # get new image from server
    for file in dir_path:
        if filenameImage == file[0]:
            newfilename = uniquify(file[0])
            ftp.retrbinary("RETR " + file[0] ,open("./" + newfilename, 'wb').write)
            new_picture = face_recognition.load_image_file("./" + newfilename)
            my_face_encoding = face_recognition.face_encodings(new_picture)[0]
    
    ftp.cwd('/KnownImagesLibrary')
    imagefolder = list(ftp.mlsd())
    
    for images in imagefolder:
        imagePath = "./" + images[0]
        ftp.retrbinary("RETR " +images[0] ,open(imagePath, 'wb').write)
        temp_picture = face_recognition.load_image_file(imagePath)
        unknown_face_encoding = face_recognition.face_encodings(temp_picture)[0]
        results = face_recognition.compare_faces([my_face_encoding], unknown_face_encoding)
        
        os.remove(imagePath)
        
        if results[0] == True:
            print("Opening Door...")
            openDoor(outcomport)
            
            ftp.quit()
            break
        else:
            print("Alert! Face Not Recognized.")
                
def ConnectToftp(FTP_HOST, FTP_USER, FTP_PASS, filenameImage,outcomport):
    ftp = ftplib.FTP(FTP_HOST, FTP_USER, FTP_PASS)
    ftp.encoding = "utf-8"
    
    uploadFTP(filenameImage,ftp)
    facialRecognitionModel(filenameImage,ftp,outcomport)
    

In [13]:
while True:
    reading = start('COM6')
    if reading == "ON":
        print("Analyzing...")
        captureimage()
        ConnectToftp("10.50.202.242","user09","user09","newphoto.jpg","COM7")

# reading = start('COM6')
# if reading == "ON":
#     print(reading)
#     captureimage()
#     ConnectToftp("10.50.202.242","user09","user09","newphoto.jpg","COM7")

Analyzing...
Alert! Face Not Recognized.
Analyzing...
Alert! Face Not Recognized.


SerialException: ClearCommError failed (PermissionError(13, 'The device does not recognize the command.', None, 22))

In [ ]:
# import serial

# ser = serial.Serial("COM7", 9600, timeout = 1, write_timeout= 1) #Change your port name COM... and your baudrate

# def retrieveData():
#     ser.write(b'1')
#     data = ser.readline().decode('ascii')
#     return data

# while(True):
#     uInput = input("Retrieve data? ")
#     if uInput == '1':
#         print(retrieveData())
#     else:
#         ser.write(b'0')